In [2]:
import pandas as pd
import numpy as np
df = pd.DataFrame.from_csv("3challenge.csv")
dftraining = df.loc[~np.isnan(df['label'])]
dftesting = df.loc[np.isnan(df['label'])]
print(dftraining.shape)
print(dftesting.shape)

(5000, 9)
(5000, 9)


In [3]:
tr = dftraining.iloc[:,:8]
tr_l = dftraining.iloc[:,8]
te = dftesting.iloc[:,:8]

In [4]:
df.var(axis = 0)/df.mean(axis = 0)

Y0       4.126357
Y1       4.186036
Y2       4.120442
Y3       4.170287
Y4       4.182009
Y5       4.164098
Y6       4.142483
Y7       4.153278
label    0.086571
dtype: float64

In [5]:
# Linear Regression


from sklearn import linear_model
linear = linear_model.LinearRegression()
model = linear.fit(tr,tr_l)
predictions = model.predict(tr)

sum((tr_l-predictions)**2)/len(tr_l)


0.00054912113345963762

In [6]:
# Ridge Regression

from sklearn import linear_model
rid = linear_model.Ridge (alpha = .5)
model = rid.fit(tr,tr_l)
predictions = model.predict(tr)

sum((tr_l-predictions)**2)/len(tr_l)

0.00054912113346195705

In [7]:
import statsmodels.api as sm


bin_model = sm.GLM(tr_l, tr, family=sm.families.Poisson()).fit()
predictions = bin_model.predict(tr)

sum((tr_l-predictions)**2)/len(tr_l)


C:\Users\user\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


0.16136756158206575

In [8]:
import statsmodels.api as sm
tr = dftraining.iloc[:,:8]

bin_model = sm.GLM(tr_l, tr, family=sm.families.Poisson()).fit()
predictions = bin_model.predict(tr)

sum((tr_l-predictions)**2)/len(tr_l)

0.16136756158206575

In [66]:
import statsmodels.api as sm


bin_model = sm.GLM(tr_l, tr, family=sm.families.NegativeBinomial()).fit()
predictions = bin_model.predict(tr)

sum((tr_l-predictions)**2)/len(tr_l)


0.17087975021207327

In [9]:
import statsmodels.api as sm


bin_model = sm.GLM(tr_l, tr, family=sm.families.Gaussian()).fit()
predictions = bin_model.predict(tr)

sum((tr_l-predictions)**2)/len(tr_l)


0.00055725226786002837

In [10]:
import statsmodels.api as sm


bin_model = sm.GLM(tr_l, tr, family=sm.families.Binomial()).fit()
predictions = bin_model.predict(tr)

sum((tr_l-predictions)**2)/len(tr_l)

0.056833027811275021

In [11]:
# Bayesian Approach w. uninformative prior
# Beta(0,0) (Beta(alpha,beta))
#
# Posterior is Beta(alpha+y, beta+n-y)
# three point estimators
# mean = (alpha+y)/(alpha+beta+n)
# median = (alpha + y - 1/3)/(alpha+beta+n-2/3)
# mode = (alpha + y - 1)/(alpha+beta+n-2)

y = tr.sum(axis = 1)
n = 320
alpha = 0
beta = 0

predictions_mean = (alpha+y)/(alpha+beta+n)
predictions_median = (alpha + y - 1/3)/(alpha+beta+n-2/3)
predictions_mode = (alpha + y - 1)/(alpha+beta+n-2)

print(sum((tr_l-predictions_mean)**2)/len(tr_l))
print(sum((tr_l-predictions_median)**2)/len(tr_l))
print(sum((tr_l-predictions_mode)**2)/len(tr_l))


0.000560901529086
0.000563250025988
0.000569823028207


In [12]:
# Bayesian Approach w. informative prior
# Beta(2,5) (Beta(alpha,beta))
#
# Posterior is Beta(alpha+y, beta+n-y)
# three point estimators
# mean = (alpha+y)/(alpha+beta+n)
# median = (alpha + y - 1/3)/(alpha+beta+n-2/3)
# mode = (alpha + y - 1)/(alpha+beta+n-2)

y = tr.sum(axis = 1)
n = 320
alpha = 2
beta = 5

predictions_mean = (alpha+y)/(alpha+beta+n)
predictions_median = (alpha + y - 1/3)/(alpha+beta+n-2/3)
predictions_mode = (alpha + y - 1)/(alpha+beta+n-2)

print(sum((tr_l-predictions_mean)**2)/len(tr_l))
print(sum((tr_l-predictions_median)**2)/len(tr_l))
print(sum((tr_l-predictions_mode)**2)/len(tr_l))

0.000549301609581
0.000549423099645
0.000551408752708


In [13]:
# It seems I got the best result with the linear rigression,
# not Poisson or negative binomial regression, so I will use it for the test data
from sklearn import linear_model
linear = linear_model.LinearRegression()
model = linear.fit(tr,tr_l)
predictions = model.predict(te)

df.iloc[5000:,8] = model.predict(te)


In [60]:
df

,Y0,Y1,Y2,Y3,Y4,Y5,Y6,Y7,label
0,0.0,3.0,1.0,3.0,0.0,3.0,2.0,0.0,0.043358
1,8.0,5.0,7.0,7.0,4.0,6.0,6.0,8.0,0.201109
2,6.0,3.0,2.0,6.0,3.0,3.0,6.0,4.0,0.136372
3,12.0,16.0,12.0,16.0,12.0,16.0,13.0,15.0,0.369622
4,5.0,4.0,5.0,5.0,4.0,4.0,7.0,4.0,0.106446
5,28.0,29.0,24.0,26.0,26.0,24.0,29.0,25.0,0.661859
6,0.0,2.0,0.0,0.0,2.0,1.0,0.0,1.0,0.033296
7,12.0,16.0,9.0,10.0,14.0,12.0,8.0,9.0,0.284306
8,4.0,3.0,12.0,7.0,8.0,4.0,3.0,6.0,0.140339
9,5.0,8.0,5.0,5.0,7.0,5.0,4.0,6.0,0.148866


In [16]:
df.to_csv("3challenge.csv")